
# ***T5 Paraphrase***

In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece transformers

import sentencepiece

In [4]:
import pandas as pd
train = pd.read_csv('reddit train.csv')

In [7]:
train = train[:5000]

In [8]:
grouped = train.groupby(train["Y"])
one_labeled_texts = grouped.get_group(1)
zero_labeled_texts = grouped.get_group(0)

In [9]:
one_labeled_texts = one_labeled_texts.reset_index(drop= True)
zero_labeled_texts = zero_labeled_texts.reset_index(drop= True)

In [14]:
import pandas as pd

seperated_data_zero = []

data = zero_labeled_texts["text"]

context = ""
for j in range(len(data)):
  seperated_data_zero.append(data[j])

print(seperated_data_zero[0])

Bring back old wits end and lets see how long your mana lasts :)


In [15]:
seperated_data_one = []

data = one_labeled_texts["text"]

context = ""
for j in range(len(data)):
  seperated_data_one.append(data[j])

print(seperated_data_one[0])

I don't know man, that Jeep looks really cool too...


In [17]:
len(seperated_data_zero)   

2513

In [18]:
len(seperated_data_one)   

2487

In [51]:
#fine-tuned model is taken from huggingface
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

In [52]:
#let's define a function that will do the paraphrase
def paraphrase(sentence):
  text =  "paraphrase: " + sentence + " </s>"
  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to("cpu"), encoding["attention_mask"].to("cpu")


  outputs = model.generate(
     input_ids=input_ids, attention_mask=attention_masks,
     max_length=256,
     do_sample=True,
     top_k=120,
     top_p=0.95,
     early_stopping=True,
     num_return_sequences=1
  )

  for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True, use_cuda =False)
  return line


In [54]:
paraphrase("This is something which i cannot understand at all")

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'This is something which i cannot understand at all.'

In [55]:
augmented_0_data=[]

for s in range(len(seperated_data_zero)):
  augmented_0_data.append(paraphrase(seperated_data_zero[s]))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [61]:
augmented_1_data=[]

for s in range(len(seperated_data_one)):
  augmented_1_data.append(paraphrase(seperated_data_one[s]))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [66]:
T5_reddit_0 = pd.DataFrame(augmented_0_data, columns = ['text'])
T5_reddit_1 = pd.DataFrame(augmented_1_data, columns = ['text'])

In [ ]:
T5_reddit_0.to_csv('T5_reddit_0.csv')
T5_reddit_1.to_csv('T5_reddit_1.csv')